# Patrones de diseño en Python
## Introducción a los patrones de diseño y qué herramientas brinda Python para implementarlos

---

### Luciano Serruya Aloisi
#### 30 de Septiembre, 2017

***

##### Según GoF, un _Patrón de Diseño_ se define como:

> _Descripciones de clases y objetos relacionados que están particularizados para resolver un problema de diseño general en un determinado contexto_

***

***
>_Each pattern describes a problem which occurs over and
over again in our environment, and then describes the core of the solution to that
problem, in such a way that you can use this solution a million times over, without ever
doing it the same way twice_

_Christopher Alexander, Sara Ishikawa, Murray Silverstein, Max Jacobson,
Ingrid Fiksdahl-King, and Shlomo Angel. A Pattern Language. Oxford
University Press, New York, 1977._

Incorporar las clases de PD (Estructurales, Creacionales, Comportamiento)

Hablar sobre First-class objects, Closures, y capaz que bounded y unbounded methods 
(aunque los unbounded methods no existen más en Python3, ahora son funciones)

Poner un PD de cada clase
Agregar memes obvio

### GoF organiza los patrones por _propósito_, y por _alcance_

### Categorías por _propósito_
* Creacionales: se encargan de la instanciación de objetos
* Comportamiento: caracterizan las formas en que los objetos y las clases interactúan entre sí y cómo distribuyen la responsabilidad
* Estructurales: se encargan de cómo se componen las clases y los objetos

### Categorías por _alcance_
* Clase: especifican relaciones entre clases y sus clases hijas. Relación estática a través de la **herencia**
* Objeto: especifican relaciones entre objetos, dichos objetos pueden ser cambiados en tiempo de ejecución. Resultados más dinámicos a través de la **composición** y **delegación**

## Conceptos de Python

### Todo es un objeto
* Todos los objetos en Python son *objetos de primera clase* (*first-class objects*)

> In programming language design, a first-class citizen (also type, object, entity, or value) in a given programming language is an entity which supports all the operations generally available to other entities. These operations typically include being passed as an argument, returned from a function, modified, and assigned to a variable.

[First-class citizen](https://en.wikipedia.org/wiki/First-class_citizen)

### Closures
* Una *clausura* (*closure*) es un objeto función que incluye tanto su código como su ambiente (*scope*) en el cual fue declarado


[Closure](http://www.bogotobogo.com/python/python_closure.php)

In [2]:
def my_map(func, l):
    """Aplica una función a todos los elementos de una lista"""
    ret = list()
    for x in l:
        ret.append(func(x))
    return ret

def square(x):
    """Devuelve el cuadrado de un número"""
    return x*x

# Números entre 1 y 10
lista = list(range(1,10))

print("lista = {}".format(lista))

# Conseguimos una nueva lista con todos los valores de 'lista' elevados al cuadrado
# Le pasamos a 'my_map' el objeto función 'square'
cuadrados = my_map(square, lista)

print("cuadrados = {}".format(cuadrados))

lista = [1, 2, 3, 4, 5, 6, 7, 8, 9]
cuadrados = [1, 4, 9, 16, 25, 36, 49, 64, 81]


In [1]:
def my_map(func, l):
    """Aplica una función a todos los elementos de una lista"""
    return [func(x) for x in l]

# Números entre 1 y 10
lista = list(range(1,10))

print("lista = {}".format(lista))

# Conseguimos una nueva lista con todos los valores de 'lista' elevados al cuadrado
# Le pasamos a 'my_map' un objeto definido en la invocacion de la funcion
cuadrados = my_map(lambda x: x*x, lista)

print("cuadrados = {}".format(cuadrados))

lista = [1, 2, 3, 4, 5, 6, 7, 8, 9]
cuadrados = [1, 4, 9, 16, 25, 36, 49, 64, 81]


### Patrones creacionales

#### Borg (monoestado)
Implementa un comportamiento *Singleton* (una única instancia de una clase). Instancia objetos distintos pero todos comparten el mismo estado

Fuente: [Borg](https://github.com/faif/python-patterns/blob/master/creational/borg.py)

In [18]:
class Borg(object):
    
    #diccionario que contiene todos los atributos del objeto.
    __shared_state = {}

    def __init__(self):
        #los miembros de datos de un objeto los podemos ver o manipular a través del atributo "__dict__"
        
        #a la nueva instancia que inicializamos, le asignamos a su diccionario propio, el diccionario compartido
        #cualquier modificación que se haga el objeto a sus atributos, se reflejará en todos las otras instancias
        self.__dict__ = self.__shared_state
        self.state = 'Init'

    def __str__(self):
        return self.state
    
    def show(self):
        print(self.__dict__)


rm1 = Borg()
rm2 = Borg()

#el id de un objeto es el dato que lo hace único a cada objeto
print('ID rm1: {0}'.format(id(rm1)))
print('ID rm2: {0}'.format(id(rm2)))

rm1.state = 'Idle'
print('rm1: {0}'.format(rm1))
print('rm2: {0}'.format(rm2))


rm2.state = 'Running'
print('rm1: {0}'.format(rm1))
print('rm2: {0}'.format(rm2))


rm2.state = 'Zombie'
print('rm1: {0}'.format(rm1))
print('rm2: {0}'.format(rm2))

rm1.show()
rm2.show()

ID rm1: 140697360973216
ID rm2: 140697360973384
rm1: Idle
rm2: Idle
rm1: Running
rm2: Running
rm1: Zombie
rm2: Zombie
{'state': 'Zombie'}
{'state': 'Zombie'}


#### Factory
Define una nueva interfaz para instanciar objetos. Evita invocar al constructor del objeto directamente.

Fuente: [Factory](https://github.com/faif/python-patterns/blob/master/creational/factory_method.py)

#### Fábrica como función

In [11]:
class GreekGetter(object):

    """A simple localizer a la gettext"""

    def __init__(self):
        self.trans = dict(dog="σκύλος", cat="γάτα")

    def get(self, msgid):
        """We'll punt if we don't have a translation"""
        return self.trans.get(msgid, str(msgid))


class EnglishGetter(object):

    """Simply echoes the msg ids"""

    def get(self, msgid):
        return str(msgid)


def get_localizer(language="English"):
    """The factory method"""
    languages = dict(English=EnglishGetter, Greek=GreekGetter)
    return languages[language]()


# Create our localizers
e, g = get_localizer(language="English"), get_localizer(language="Greek")
# Localize some text
for msgid in "dog parrot cat bear".split():
    print("{} -> {}".format(e.get(msgid), g.get(msgid)))

dog -> σκύλος
parrot -> parrot
cat -> γάτα
bear -> bear


#### Fábrica como clase

También podríamos ocultar la definición de las clases que crea la fábrica dentro de la clase/función fábrica; de esta manera ocultamos las clases que instancia la fábrica a sus clientes.

[Fábrica como clase](https://github.com/gennad/Design-Patterns-in-Python/blob/master/factory.py)

In [7]:
class Pizza(object):
    def __init__(self):
        self._price = None

    def get_price(self):
        return self._price

class HamAndMushroomPizza(Pizza):
    def __init__(self):
        self._price = 8.5

class DeluxePizza(Pizza):
    def __init__(self):
        self._price = 10.5

class HawaiianPizza(Pizza):
    def __init__(self):
        self._price = 11.5

class PizzaFactory(object):
    @staticmethod
    def create_pizza(pizza_type):
        if pizza_type == 'HamMushroom':
            return HamAndMushroomPizza()
        elif pizza_type == 'Deluxe':
            return DeluxePizza()
        elif pizza_type == 'Hawaiian':
            return HawaiianPizza()

pizzas = ('HamMushroom', 'Deluxe', 'Hawaiian')
for pizza_type in pizzas:
      print('Price of {0} is {1}'.format(pizza_type, PizzaFactory.create_pizza(pizza_type).get_price()))

Price of HamMushroom is 8.5
Price of Deluxe is 10.5
Price of Hawaiian is 11.5


#### Fábrica abstracta
El objeto que utiliza la fábrica recibe una fábrica en el constructor.

Podemos tener distintas instancias que utilicen distintas fábricas según su comportamiento, o cambiar dichas fábricas en tiempo de ejecución.

[Fábrica abstracta](http://python-3-patterns-idioms-test.readthedocs.io/en/latest/Factory.html)

In [13]:
# Factory/Games.py
# An example of the Abstract Factory pattern.

class Obstacle:
    def action(self): pass

class Character:
    def interactWith(self, obstacle): pass

class Kitty(Character):
    def interactWith(self, obstacle):
        print("Kitty has encountered a {}".format(obstacle.action()))

class KungFuGuy(Character):
    def interactWith(self, obstacle):
        print("KungFuGuy now battles a {}".format(obstacle.action()))

class Puzzle(Obstacle):
    def action(self):
        return "Puzzle"

class NastyWeapon(Obstacle):
    def action(self):
        return "NastyWeapon"

# The Abstract Factory:
class GameElementFactory:
    def makeCharacter(self): pass
    def makeObstacle(self): pass

# Concrete factories:
class KittiesAndPuzzles(GameElementFactory):
    def makeCharacter(self): return Kitty()
    def makeObstacle(self): return Puzzle()

class KillAndDismember(GameElementFactory):
    def makeCharacter(self): return KungFuGuy()
    def makeObstacle(self): return NastyWeapon()

class GameEnvironment:
    def __init__(self, factory):
        self.factory = factory
        self.p = self.factory.makeCharacter()
        self.ob = self.factory.makeObstacle()
    def play(self):
        self.p.interactWith(self.ob)

g1 = GameEnvironment(KittiesAndPuzzles())
g2 = GameEnvironment(KillAndDismember())
g1.play()
g2.play()


Kitty has encountered a Puzzle
KungFuGuy now battles a NastyWeapon


In [27]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from __future__ import print_function
import time


class SalesManager:
    def talk(self):
        print("Sales Manager ready to talk")


class Proxy:
    def __init__(self):
        self.busy = 'No'
        self.sales_manager = None

    def talk(self):
        print("PROXY CHECKING FOR SALES MANAGER AVAILABILITY")
        if self.busy == 'No':
            self.sales_manager = SalesManager()
            time.sleep(0.1)
            self.sales_manager.talk()
        else:
            time.sleep(0.1)
            print("Sales Manager is busy")


class NoTalkProxy(Proxy):
    def talk(self):
        print("PROXY CHECKING FOR SALES MANAGER AVAILABILITY")
        time.sleep(0.1)
        print("This Sales Manager will not talk to you",
              "whether he/she is busy or not")


p = Proxy()
p.talk()
p.busy = 'Yes'
p.talk()


p = NoTalkProxy()
p.talk()
p.busy = 'Yes'
p.talk()

PROXY CHECKING FOR SALES MANAGER AVAILABILITY
Sales Manager ready to talk
PROXY CHECKING FOR SALES MANAGER AVAILABILITY
Sales Manager is busy

Is COSO busy? => No

PROXY CHECKING FOR SALES MANAGER AVAILABILITY
This Sales Manager will not talk to you whether he/she is busy or not

Is COSO busy? => Yes

PROXY CHECKING FOR SALES MANAGER AVAILABILITY
This Sales Manager will not talk to you whether he/she is busy or not


## Implementado ITERADORES

In [30]:
class Colectivo:
    def __init__(self, max_pj):
        self._cant_pasajeros = 0
        self._max_pasajeros = max_pj
        self._pasajeros = list()
        self.__cont = 0
        
    def __iter__(self):
        return self
    
    def set_pasajeros(self, pasajeros):
        self._pasajeros = pasajeros
        self._cant_pasajeros = len(pasajeros)
        
    def __next__(self):
        if self.__cont == self._cant_pasajeros:
            self.__cont = 0
            raise StopIteration()
            
        self.__cont += 1
        return self._pasajeros[self.__cont - 1]    
            
cole = Colectivo(10)

pasajeros = ["juan", 'pedro', 'pepe', 'luciano', 'pablo']

cole.set_pasajeros(pasajeros)
# next(cole)

for p in cole:
    print(p)

it = iter(cole)

# for p in cole:
#     print(p)



juan
pedro
pepe
luciano
pablo


In [29]:
class Colectivo:
    def __init__(self, max_pj):
        self._cant_pasajeros = 0
        self._max_pasajeros = max_pj
        self._pasajeros = list()
        self.__cont = 0
        
    
    def set_pasajeros(self, pasajeros):
        self._pasajeros = pasajeros
        self._cant_pasajeros = len(pasajeros)
        
        
    def __iter__(self):
        return iter(self._pasajeros)
    
    #como el colectivo devuelve el iterador de la lista, el metodo __next__ esta demas en la clase Colectivo
#     def __next__(self):
#         if self.__cont < self._cant_pasajeros:
#             self.__cont += 1
#             return self._pasajeros[self.__cont - 1]
#         else:
#             self.__cont = 0
#             raise StopIteration()
            
            
cole = Colectivo(10)

pasajeros = ["juan", 'pedro', 'pepe', 'luciano', 'pablo']

cole.set_pasajeros(pasajeros)


for p in cole:
    print(p)

juan
pedro
pepe
luciano
pablo
